# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 **이전 처리결과를 입력으로 받아 처리한 후 다음 단계로 결과를 전달**한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행등을 순차적으로 연결**하여 실행 할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 이러한 체인구조를 사용하면 **작업흐름이 명확**해지고 **코드의 재사용성**이 높아지며 **유지 보수 및 확장성이** 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 각 클래스는 다양한 chain 알고리즘들을 미리 구현한 것으로 상황에 맞는 것을 선택하여 필요한 구성요소를 전달해 생생한다.
- 이 방식은 LangChain의 **초기 방식**이며, 새로운 기능 확장이나 유연한 구성에 한계가 있기 때문에 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.
  - 현재 LangChain에서는 권장하지 않는 방식이다.

### 2. LCEL (LangChain Expression Language) 방식

- LCEL은 체인을 표현식(Expression) 기반의 선언적 파이프라인 방식으로 구성할 수 있도록 설계된 최신 체인 구성 방법이다. 
- 각 컴포넌트들을 `|` 연산자로 연결하여, 흐름이 자연스럽게 이어지는 형태의 체인을 구성한다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 융통성과 확장성 있는 체인 구성**이 가능하다.
- LCEL은
  - 선형적 흐름 구조를 가진다.
  - 문법이 간결하고 선언적이다.
  - 체인의 구조가 코드만 봐도 쉽게 파악된다.
  - 유연하고 확장성이 매우 뛰어나다.
- `Runnable` 기반 구조
  - LCEL방식을 구성하는 모든 컴포넌트들은 `Runnable` 이라는 공통 인터페이스를 기반으로 동작한다.
  - 체인을 구성하는 각 컴포넌트들은 `Runnable` 을 상속하여 구현하여 이를 통해 일관된 실행 인터페이스를 제공한다.
  - **공통 메소드**:
    - `invoke()`: 단일 입력에 대한 처리
    - `batch()`: 다수 입력을 묶어서 한번에 처리
    - `stream()`: 스트리밍 방식의 요청
    - `ainvoke()`, `abatch()`, `astream()`: 비동기적 처리 메소드

In [1]:
# 기존 off the shelf 방식
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
prompt = ChatPromptTemplate.from_template(
    template="{item}에 어울리는 브랜드 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# input_variable->(prompt)->query->(model)->답변->(parser) -> 최종답변

In [2]:
# 기존 off the shelf 방식
from langchain_classic import LLMChain
# chain을 구성하는 요소들을 넣어서 생성.
chain = LLMChain( # prompt->model->parser 기본체인을 구성.
    prompt=prompt,
    llm=model,
    output_parser=parser
)
 # 첫번째: Prompt -> Prompt에 전달할 값
res = chain.invoke({"item":"가방", "count":3})
print(res)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_14208\2438020229.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain( # prompt->model->parser 기본체인을 구성.


{'item': '가방', 'count': 3, 'text': '다음 세 가지 제안드립니다.\n\n1) 담다 (DAMDA) — 담는다는 뜻으로 실용적이고 미니멀한 데일리백 브랜드에 어울립니다.  \n2) 채움 (CHAEUM) — 채움/충만의 이미지로 고급 가죽이나 디자인 중심의 프리미엄 라인에 적합합니다.  \n3) 바람길 (BARAMGIL) — 여행·아웃도어 감성의 경쾌한 브랜드(토트·크로스·백팩)에 잘 맞습니다.'}


In [3]:
print(res['text'])

다음 세 가지 제안드립니다.

1) 담다 (DAMDA) — 담는다는 뜻으로 실용적이고 미니멀한 데일리백 브랜드에 어울립니다.  
2) 채움 (CHAEUM) — 채움/충만의 이미지로 고급 가죽이나 디자인 중심의 프리미엄 라인에 적합합니다.  
3) 바람길 (BARAMGIL) — 여행·아웃도어 감성의 경쾌한 브랜드(토트·크로스·백팩)에 잘 맞습니다.


In [4]:
from langchain_core.runnables import RunnableSequence

a = RunnableSequence(prompt , model , parser)
a.invoke({"item":"TV 브랜드", "count":1})

"브랜드 이름: 루미온 (LumiOn)\n\n짧은 설명: '빛(Lumi)'과 '켜다(On)'를 합쳐 선명하고 생동감 있는 화면 경험을 강조한 TV 브랜드 이름입니다."

In [2]:
# LCEL
chain2 = prompt | model | parser
print(type(chain2))
res2 = chain2.invoke({"item":"TV 브랜드", "count":3}) 
# 첫번째 구성요소(prompt)에 전달할 값을 전달.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [3]:
print(res2)

아래 3가지 이름을 제안드립니다. 각 이름에 짧은 의미와 태그라인도 함께 적었습니다.

1) 오로라뷰 (AuroraView)  
- 의미: 오로라처럼 풍부한 색감과 자연스러운 화질을 연상시키는 프리미엄 이미지  
- 태그라인 예: "생생한 컬러, 눈에 담다"

2) 루미센스 (LumiSense)  
- 의미: 빛(Lumi)과 센스(Sense)를 결합한 이름으로, 자동 밝기·화질 최적화 같은 스마트 기능 강조  
- 태그라인 예: "스마트한 빛, 더 선명한 화면"

3) 비비온 (Vivion)  
- 의미: 'Vivid'와 'Vision' 계열의 어감을 살려 활기차고 친근한 화질 브랜드 이미지  
- 태그라인 예: "눈에 띄는 생생함"

원하시면 타깃(프리미엄/대중/젊은 층)별 추가 아이디어나 로고·도메인 가용성까지 검토해 드리겠습니다. 어느 스타일을 더 원하시나요?


# Runnable 타입 주요 클래스

## [Runnable](https://reference.langchain.com/python/langchain_core/runnables/#langchain_core.runnables.base.Runnable)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - **`invoke(input, config:RunnableConfig)->output`**: 단일 입력을 처리하여 결과를 반환.
    - **`batch(input:list, config:RunnableConfig|list[RunnableConfig]) -> list[Output]`**: 여러 입력 데이터들을 한 번에 처리.
    - **`stream(input, config:RunnableConfig) -> Iterator[Output]`**: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - **`assign(**kwargs)`**:
      -  앞 Runnable의 출력 결과에 새로운 key–value 쌍의 Field 추가(assign) 하여 다음 Runnable로 전달.
      -  값으로는 Runnable 객체(LCEL체인등)나 고정 값(리터럴) 모두 가능하며, 각 항목은 실행 시 평가되어 기존 출력에 병합한다.
      -  주로 앞 단계의 출력에 부가 정보(field)를 추가하고자 할 때 사용한다. 특히 `RunnablePassthrough`와 결합해, 입력을 그대로 넘기면서 특정 field만 추가할 때 자주 사용
### Runnable의 주요 구현체(하위 클래스)

- **`RunnableSequence`**
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - 보통은 LCEL 문법을 사용해서 정의한다.
      - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
  
-  **`RunnablePassthrough`**
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
      - 앞 Runnable으로 부터 전달 받은 **입력 값을 다음 Runnable로 그대로 전달**한다.
           - `RunnablePassthrough()`
      - 입력받은 값에 **Field를 추가**해서 전달할 경우 `assign()` 메소드를 사용한다.
           - `RunnablePassthrough.assign(new_key1="new_value1", new_key2="new_value2", ..)`

- **`RunnableParallel`**
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 
        ```python
        RunnableParallel(
            {
                "key1":Runnable1, 
                "key2":Runnable2,
                "key3":Runnable3, ...
            }
        )
        ```
      - 각 Runnable의 실행결과를 Value로 Dictionary를 생성해서 반환한다.
      - LCEL로 정의할 때는 Chain에 dictionary로 정의한다.

- **`RunnableLambda`**
    - 일반 함수를 `Runnable`로 변환할 때 사용한다.
    
    - Runnable을 입력해야 하는 자리에 함수를 넣어야 하는 경우 RunnableLambda로 그 함수를 Runnable로 만들어 넣는다.
    - **구현**
      1. `RunnableLambda(함수객체)`
      2. `@chain` decorator를 이용해 함수를 `RunnableLambda`로 구현할 수있다.
      3. LCEL 에 함수를 포함시키면 `RunnableLambda`로 자동 변환된다.

#### Runnable 예제

In [13]:
from langchain_core.runnables import Runnable

class MyRunnable(Runnable):
    # invoke(), stream(), batch(), assign() 중에서 필요한 메소드를 재정의.
    def invoke(self, input_data, config=None):
        # input_data: 처리를 위한 입력값. 1개
        # config: RunnableConfig -> 입력시에는 dict
        # Runnable이 실행할 때 필요한 설정정보를 입력.
        #   - {"configurable":{key: value}}
        output = ""
        if config is not None:
            if config['configurable'].get("lang") == "en":
                output = "\n\n답변은 영어로 해주세요."
        
        return f"{input_data}에 대해서 한 문장으로 정의해주세요.{output}"


In [14]:
mr1 = MyRunnable()
print(mr1.invoke('사과'))
print(mr1.invoke("컴퓨터", {"configurable":{"lang":"en"}}))

사과에 대해서 한 문장으로 정의해주세요.
컴퓨터에 대해서 한 문장으로 정의해주세요.

답변은 영어로 해주세요.


In [16]:
# chain
chain = mr1 | model
res = chain.invoke("사과", {"configurable":{"lang":"en"}})
print(res.content)

An apple is a round, typically red, green, or yellow edible fruit produced by the Malus domestica tree, valued for its sweet-to-tart flavor and crisp texture.


#### RunnableLambda 예제

In [17]:
from langchain_core.runnables import RunnableLambda

mr2 = RunnableLambda(lambda input_data : f"{input_data}를 한문장으로 설명해줘.")
isinstance(mr2, Runnable)

True

In [18]:
mr2.invoke("LLM")

'LLM를 한문장으로 설명해줘.'

In [ ]:
def sum(input_data):
    return input_data + 100

RunnableLambda(sum).invoke(1000)

1100

In [ ]:
# Runnable로 만들 함수
# 파라미터: 1개
# 반환값  : 다음 체인구성에 전달할 값의 형식.

In [20]:
prompt = ChatPromptTemplate.from_template(
    template="{subject}에 대해 100글자 이내로 설명해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# 출력결과, 글자수
chain = prompt | model | parser | \
        RunnableLambda(lambda input_data: (input_data, len(input_data)))
res = chain.invoke({"subject":"AI"})

In [22]:
print(res)
len(res[0])

('인공지능(AI)은 데이터와 알고리즘으로 학습해 판단·예측·자동화 작업을 수행하는 기술입니다.', 51)


51

In [ ]:
def get_length(value):
    return value, len(value)

chain2 = prompt | model | parser | get_length
res = chain2.invoke("크리스마스")
res

('크리스마스는 예수 그리스도의 탄생을 기념하는 12월 25일의 기독교 축일로, 가족 모임·선물·트리 장식 등 다양한 문화적 전통이 있다.',
 75)

#### RunnablePassThrough 예제

In [26]:
from langchain_core.runnables import RunnablePassthrough

rp = RunnablePassthrough()
res = rp.invoke("안녕하세요")
res = rp.invoke({"a":10, "b":20})
res = rp.invoke([1, 2, 10, 20, 30])
print(res)

[1, 2, 10, 20, 30]


In [ ]:
# 입력받은 값에 field를 추가해서 전달. 입력: dict, item을 추가해서 전달

address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x: "010-1234-5678")

rp2 = RunnablePassthrough.assign(
    address=address_runnable, phone=phone_runnable
    # 변수=Runnable(Callable) key:변수-Value:Runnable반환값을 input dict에추가
)
res = rp2.invoke({"name":"홍길동"})
res

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1234-5678'}

#### RunnableParallel 예제

In [ ]:
from langchain_core.runnables import RunnableParallel

run1 = RunnableLambda(lambda x : x + 10)
run2 = RunnableLambda(lambda x : x * 10)
run3 = RunnableLambda(lambda x : x ** 10)

parallel = RunnableParallel(
    {
        "value1":run1, # run1의 실행결과를 "value1" key에 담아서 반환.
        "value2":run2,
        "value3":run3,
        "value4":RunnablePassthrough() # value4:입력값
    }
)
res = parallel.invoke(5)
res

{'value1': 15, 'value2': 50, 'value3': 9765625, 'value4': 5}

In [ ]:
# chain = RunnableLambda(lambda x : x['v2']) | parallel
from operator import itemgetter
chain = itemgetter('v1') | parallel
# itemgetter('v1')({"v1":10, "v2":20})

res = chain.invoke({"v1":10, "v2":20})
res

{'value1': 20, 'value2': 100, 'value3': 10000000000, 'value4': 10}

In [ ]:
from operator import itemgetter

ig = itemgetter('v1')  # 자료구조에서 "v1" 값을 조회.
d = {"v1":10, "v2":20}
ig(d)




ig2 = itemgetter(2)  # 자료구조[2] 조회한 결과를 반환.
ig2([1, 2, 3, 4])
ig2((10, 20, 30, 40, 50))
ig2({1:'a', 2:'c'})

'c'

In [42]:
# LCEL Chain안에서 RunnableParallel 표현식. {key:Runnable}
chain = run1 | {'result1':run2, "result2":run3}
chain.invoke(20)

{'result1': 300, 'result2': 590490000000000}

In [ ]:
# {} 와 Runnable이 `|` 로 묶이면 {}이 RunnableParellel로 변환된다.
chain = {'result1':run2, "result2":run3} | RunnablePassthrough()
chain.invoke(2)

{'result1': 20, 'result2': 1024}

In [ ]:
rp = {'result1':run2, "result2":run3}  # dict.  
rp.invoke(2)

AttributeError: 'dict' object has no attribute 'invoke'

### LCEL Chain 예제

In [ ]:
# 입력 : 음식 이름
# 출력 : 음식의 레시피
# chain: prompt_template -> model(gpt-5-mini) -> StrOutputParser

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model='gpt-5-mini')
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template(
    template="""# Instruction
당신은 숙련된 요리 전문 assistant 입니다. 
요청한 음식의 레시피를 자세하게 작성해 주세요.

# Input Data
음식이름: {food}

# Output Indicator
- markdown 형식으로 작성한다.
- 다음 항목들을 넣어 레시피를 작성한다.
    - 요리 이름
    - 요리 기본 정보
        - 난이도
        - 조리시간
        - 인분
    - 재료
    - 요리 방법
    - 팁
"""
)

recipe_chain = prompt | model | parser

In [ ]:
resp = recipe_chain.invoke({"food":"김치찌게"})

In [8]:
print(resp)

# 김치찌개

## 요리 기본 정보
- 난이도: 초중급 (재료 손질과 불 조절이 중요)
- 조리시간: 준비 10분 / 조리 25~35분 (총 35~45분)
- 인분: 3~4인분

## 재료
- 신김치(배추김치) 300~400g (익은 김치 권장)
- 김치국물(있으면) 100~150ml
- 돼지고기(삼겹살 또는 목살) 200g — 한입 크기로 썰기  
  (선택: 참치캔 1개로 대체 가능)
- 두부 1모(약 300g) — 2cm 두께로 큼직하게 썰기
- 양파 1/2개 — 채썰기
- 대파 1대 — 어슷썰기
- 다진 마늘 1큰술
- 고춧가루 1~2큰술 (매운맛 조절)
- 고추장 1큰술 (선택사항, 감칠맛과 진한 색감을 위해)
- 국간장 또는 진간장 1큰술 (간 조절용)
- 액젓 또는 멸치액젓 1작은술 (감칠맛)
- 설탕 1작은술 (김치 산미 조절용, 선택)
- 물 또는 육수 600~800ml (종이컵 3~4컵)
  - 육수 추천: 멸치+다시마 육수 (멸치 10마리 + 다시마 5×5cm 한 장으로 15분 끓여 건더기 제거)
- 식용유 또는 참기름 1큰술
- (선택) 표고·팽이버섯 등 버섯류 약간
- (선택) 청양고추 1~2개 — 어슷썰기
- 소금/후추 약간

## 요리 방법
1. 준비
   - 김치는 먹기 좋은 크기로 자르고, 김치국물이 있으면 따로 준비합니다.
   - 돼지고기는 한입 크기로 썰고 소금/후추로 가볍게 밑간(선택).
   - 두부는 물기를 빼고 큼직하게 썹니다.
   - 대파와 양파, 청양고추는 썰어 둡니다.
   - 육수가 없다면 물에 멸치·다시마로 간단히 육수를 준비(끓기 직전 다시마 빼고 10~15분 더 끓인 뒤 멸치 건더기 제거).

2. 돼지고기 볶기 (약 3~5분)
   - 냄비(뚝배기 권장)에 식용유 1큰술을 두르고 중불로 예열합니다.
   - 돼지고기를 넣어 겉면이 익고 기름이 나오도록 볶습니다. (삼겹살이면 기름이 충분히 나오므로 볶음만으로도 감칠맛이 납니다.)
   - 다진 마늘을 넣고 향이 올라올 때까지 볶습니다.

3. 김치 넣고

In [9]:
resp2 = ""
for token in recipe_chain.stream({"food":"봉골레 파스타"}):
    print(token, end="")
    resp2 += token

# 봉골레 파스타 (Spaghetti alle Vongole)

## 요리 기본 정보
- 난이도: 중간  
- 조리시간: 약 30분 (조개 해감 시간 제외: 30~60분 권장)  
- 인분: 2인분

---

## 재료
- 스파게티 200 g  
- 바지락(봉골레) 300–400 g (껍데기 있는 생조개)  
- 엑스트라 버진 올리브오일 3 큰술  
- 마늘 3–4쪽 (얇게 편썰기)  
- 건고추 1개 (또는 레드페퍼 플레이크 1/2 작은술, 취향에 따라 조절)  
- 화이트 와인 100 ml (없으면 소량의 조개육수나 물 사용 가능)  
- 소금 적당량 (파스타 삶을 물에 충분히)  
- 후추 약간  
- 파슬리(잎) 한 줌 (다진 것, 장식 및 향)  
- 레몬 1/2개 (즙, 선택 사항)  
- 버터 10 g (선택 사항: 소스가 부드러워짐)  
- (선택) 체로 걸러낸 조개 우린 국물 약 1/2컵 — 해감을 한 물 일부를 이용해 체로 걸러 사용 가능

알레르기: 조개/갑각류 알레르기 주의

---

## 요리 방법
1. 조개 해감 및 준비  
   - 바지락은 뻘과 모래 제거를 위해 찬물에 굵은 소금(바닷물 농도와 비슷하게, 약 3.5%)을 만들어 30~60분 정도 담가 해감한다. (흙 배출을 위해 가끔 저어줌)  
   - 해감 후 흐르는 물에 여러 번 헹궈 껍데기 표면의 이물질을 제거한다. 물색이 탁하면 한두 번 더 갈아준다.

2. 파스타 삶기 준비  
   - 큰 냄비에 물을 넣고 소금을 충분히 (물 1L당 10 g 정도 권장) 넣어 끓인다.  
   - 삶은 물은 파스타 소스 조절을 위해 1컵(약 240 ml) 따로 덜어둔다(리저브드 파스타 워터).

3. 조개 스튜 시작  
   - 큰 팬(또는 깊은 프라이팬)에 올리브오일을 두르고 중약불로 가열한다.  
   - 마늘과 건고추를 넣고 마늘이 투명해질 때까지(갈색으로 타지 않게) 천천히 향을 낸다.

4. 조개 익히기  
   - 팬이 달궈지면 해감한 바지락을 넣고 센 불로 올린다.  
   

In [10]:
print(resp2)

# 봉골레 파스타 (Spaghetti alle Vongole)

## 요리 기본 정보
- 난이도: 중간  
- 조리시간: 약 30분 (조개 해감 시간 제외: 30~60분 권장)  
- 인분: 2인분

---

## 재료
- 스파게티 200 g  
- 바지락(봉골레) 300–400 g (껍데기 있는 생조개)  
- 엑스트라 버진 올리브오일 3 큰술  
- 마늘 3–4쪽 (얇게 편썰기)  
- 건고추 1개 (또는 레드페퍼 플레이크 1/2 작은술, 취향에 따라 조절)  
- 화이트 와인 100 ml (없으면 소량의 조개육수나 물 사용 가능)  
- 소금 적당량 (파스타 삶을 물에 충분히)  
- 후추 약간  
- 파슬리(잎) 한 줌 (다진 것, 장식 및 향)  
- 레몬 1/2개 (즙, 선택 사항)  
- 버터 10 g (선택 사항: 소스가 부드러워짐)  
- (선택) 체로 걸러낸 조개 우린 국물 약 1/2컵 — 해감을 한 물 일부를 이용해 체로 걸러 사용 가능

알레르기: 조개/갑각류 알레르기 주의

---

## 요리 방법
1. 조개 해감 및 준비  
   - 바지락은 뻘과 모래 제거를 위해 찬물에 굵은 소금(바닷물 농도와 비슷하게, 약 3.5%)을 만들어 30~60분 정도 담가 해감한다. (흙 배출을 위해 가끔 저어줌)  
   - 해감 후 흐르는 물에 여러 번 헹궈 껍데기 표면의 이물질을 제거한다. 물색이 탁하면 한두 번 더 갈아준다.

2. 파스타 삶기 준비  
   - 큰 냄비에 물을 넣고 소금을 충분히 (물 1L당 10 g 정도 권장) 넣어 끓인다.  
   - 삶은 물은 파스타 소스 조절을 위해 1컵(약 240 ml) 따로 덜어둔다(리저브드 파스타 워터).

3. 조개 스튜 시작  
   - 큰 팬(또는 깊은 프라이팬)에 올리브오일을 두르고 중약불로 가열한다.  
   - 마늘과 건고추를 넣고 마늘이 투명해질 때까지(갈색으로 타지 않게) 천천히 향을 낸다.

4. 조개 익히기  
   - 팬이 달궈지면 해감한 바지락을 넣고 센 불로 올린다.  
   

In [ ]:
# TODO
## 입력: 번역할 내용, 언어
## 출력: "번역할 내용"을 "언어" 로 번역한 결과

# 입력: {"content":"안녕하세요", "language":"영어"}
# 출력: How are you?

In [11]:
prompt_trans = ChatPromptTemplate.from_template(
    template="""# Instruction
당신은 다국어가 가능한 숙련된 번역 Assistant입니다.
요청된 문서의 내용을 지정된 언어로 번역해 주세요.

# Input Data
- 번역할 내용: {content}
- 번역할 언어: {language}    
"""
)
translate_chain = prompt_trans | model | parser

In [12]:
resp = translate_chain.invoke(
    {"content":"안녕하세요.", "language":"스페인어"}
)

In [13]:
resp

'Hola.'

In [14]:
translate_chain.invoke(
    {"content":"Hola..", "language":"중국어"}
)

'你好..'

In [16]:
resp3 = translate_chain.invoke(
    {"content":resp2, "language":"영어"}
)

In [19]:
print(resp2)

# 봉골레 파스타 (Spaghetti alle Vongole)

## 요리 기본 정보
- 난이도: 중간  
- 조리시간: 약 30분 (조개 해감 시간 제외: 30~60분 권장)  
- 인분: 2인분

---

## 재료
- 스파게티 200 g  
- 바지락(봉골레) 300–400 g (껍데기 있는 생조개)  
- 엑스트라 버진 올리브오일 3 큰술  
- 마늘 3–4쪽 (얇게 편썰기)  
- 건고추 1개 (또는 레드페퍼 플레이크 1/2 작은술, 취향에 따라 조절)  
- 화이트 와인 100 ml (없으면 소량의 조개육수나 물 사용 가능)  
- 소금 적당량 (파스타 삶을 물에 충분히)  
- 후추 약간  
- 파슬리(잎) 한 줌 (다진 것, 장식 및 향)  
- 레몬 1/2개 (즙, 선택 사항)  
- 버터 10 g (선택 사항: 소스가 부드러워짐)  
- (선택) 체로 걸러낸 조개 우린 국물 약 1/2컵 — 해감을 한 물 일부를 이용해 체로 걸러 사용 가능

알레르기: 조개/갑각류 알레르기 주의

---

## 요리 방법
1. 조개 해감 및 준비  
   - 바지락은 뻘과 모래 제거를 위해 찬물에 굵은 소금(바닷물 농도와 비슷하게, 약 3.5%)을 만들어 30~60분 정도 담가 해감한다. (흙 배출을 위해 가끔 저어줌)  
   - 해감 후 흐르는 물에 여러 번 헹궈 껍데기 표면의 이물질을 제거한다. 물색이 탁하면 한두 번 더 갈아준다.

2. 파스타 삶기 준비  
   - 큰 냄비에 물을 넣고 소금을 충분히 (물 1L당 10 g 정도 권장) 넣어 끓인다.  
   - 삶은 물은 파스타 소스 조절을 위해 1컵(약 240 ml) 따로 덜어둔다(리저브드 파스타 워터).

3. 조개 스튜 시작  
   - 큰 팬(또는 깊은 프라이팬)에 올리브오일을 두르고 중약불로 가열한다.  
   - 마늘과 건고추를 넣고 마늘이 투명해질 때까지(갈색으로 타지 않게) 천천히 향을 낸다.

4. 조개 익히기  
   - 팬이 달궈지면 해감한 바지락을 넣고 센 불로 올린다.  
   

In [17]:
print(resp3)

Spaghetti alle Vongole (Vongole Pasta)

Basic information
- Difficulty: Medium
- Cooking time: About 30 minutes (excluding clam purging time: 30–60 minutes recommended)
- Servings: 2

Ingredients
- Spaghetti 200 g
- Clams (vongole) 300–400 g (live clams in shell)
- Extra virgin olive oil 3 tablespoons
- Garlic 3–4 cloves (thinly sliced)
- Dried chili pepper 1 (or 1/2 teaspoon red pepper flakes, adjust to taste)
- White wine 100 ml (if unavailable, a small amount of clam broth or water can be used)
- Salt as needed (make the pasta cooking water well salted)
- Black pepper to taste
- Parsley leaves, a handful (chopped, for garnish and flavor)
- Lemon 1/2 (juice, optional)
- Butter 10 g (optional — makes the sauce silkier)
- (Optional) About 1/2 cup strained clam water — you can use some of the purging water, strained through a sieve

Allergy: Contains shellfish/crustacean — take care if allergic.

Method
1. Purge and prepare the clams
   - To remove grit and sand, soak the clams in cold 

### Chain과 Chain간의 연결

In [20]:
type(recipe_chain)

langchain_core.runnables.base.RunnableSequence

In [ ]:
# recipe_chain - translate_chain (연결)
# 음식 레시피를 원하는 언어로 출력.
# recipe_chain: 입력 - food:음식이름
# translate_chain: 입력 - content: 번역할내용, language: 언어

In [ ]:
from operator import itemgetter
# chain입력: {food:음식이름, language: 언어}
# 첫번째 chain: food
# language는 두번째 chain으로 넘겨야 한다.
# 첫번째 chain의 출력결과를 content로 두번째 chain으로 넘긴다.
chain = {
    "content":recipe_chain,
    "language":itemgetter("language")
} | translate_chain

result = chain.invoke({"food":"된장찌게", "language":"영어"})

SyntaxError: expression expected after dictionary key and ':' (697917488.py, line 11)

In [25]:
# print(result)
from IPython.display import Markdown
Markdown(result)

# Doenjang-jjigae (Korean Soybean Paste Stew)

## Basic Information
- Difficulty: Easy
- Cooking time: about 30 minutes (about 40 minutes including stock preparation)
- Servings: 3–4

---

## Ingredients
- Water or stock: 800 ml – 1 L
  - (Basic) Anchovy-kelp stock: 10 dried anchovies, 1 piece of kelp (about 8×8 cm) + 1 L water
  - Or you can use 800 ml rice-rinse water (yields a nuttier flavor)
- Doenjang (Korean soybean paste): 2.5 – 3 tablespoons (rice-scoop tablespoon)
- Gochujang (optional): 1 teaspoon (for extra umami and color)
- Potato: 1 medium (about 200 g) — cut into cubes
- Korean zucchini (aehobak): 1/2 — diagonally sliced
- Onion: 1/2 — sliced or roughly cut
- Tofu: 1/2 block (about 150–200 g) — cut into cubes
- Shiitake or oyster mushrooms (optional): 2–3 — sliced
- Scallion (green onion): 1 stalk — finely sliced
- Cheongyang chili (optional): 1–2 — diagonally sliced (or use other hot green chili)
- Minced garlic: 1 teaspoon
- Salt or soup soy sauce (guk-ganjang): a little (to adjust seasoning)
- Sesame oil (optional): 1 teaspoon
- Perilla seed powder (optional): 1–2 tablespoons (for a nuttier taste)

Optional (seafood/meat)
- 150–200 g seafood such as clams, clam meat, or shrimp, OR
- 100 g thinly sliced pork belly

---

## Method

1. Prepare the stock
   - In a pot, add the dried anchovies, kelp, and 1 L water and heat over medium.
   - Just before the water boils (about 3–5 minutes), remove the kelp. Continue to boil the anchovies for another 5–7 minutes until impurities appear, then strain to use only the stock.
   - To make it quicker, you can use 800 ml rice-rinse water or store-bought stock, or just plain water.

2. Dissolve the doenjang
   - For a clean broth, press the doenjang through a fine sieve directly into the stock so solids remain in the sieve.
   - Alternatively, take 2–3 tablespoons of hot broth into a small bowl, dissolve the doenjang there, and then return it to the pot to ensure it mixes evenly.

3. Cook potatoes → onion
   - Bring the stock to a simmer and add the potatoes first; cook over medium heat for 5–7 minutes until the potatoes are nearly tender (potatoes take longer to cook).
   - When the potatoes are about half done, add the onion and shiitake mushrooms and simmer 2–3 minutes more.

4. Add zucchini and tofu
   - Add the zucchini and simmer about 2 minutes, then add the tofu. Be careful not to break the tofu.
   - If using, add the minced garlic and simmer for another 2–3 minutes.

5. Add seafood/meat (optional)
   - If using clams or shrimp, add them at this stage and simmer until cooked.
   - If using pork belly, it’s better to lightly fry it first to render some fat, then add the stock and proceed—this adds extra savory depth.

6. Adjust seasoning and finish
   - When all ingredients are cooked, lower the heat and taste. If needed, adjust seasoning with a little salt or soup soy sauce.
   - Add the sliced chili and scallion and cook only about 1 more minute.
   - Just before turning off the heat, stir in 1 teaspoon sesame oil and 1–2 tablespoons perilla seed powder for extra nuttiness (optional).

7. Serve
   - Serve hot with rice. Goes well with kimchi or kkakdugi (radish kimchi).

---

## Tips
- Choosing doenjang: Commercial store-bought doenjang tends to be saltier; homemade doenjang has deeper flavor. If it’s too salty, use less and adjust seasoning with the ingredients and broth.
- Dissolving doenjang: Straining the doenjang through a sieve gives a cleaner broth with fewer bits. Stirring it straight into the pot can leave more residue and a slightly stronger, muddier taste.
- Rice-rinse water: Using rice-rinse water as the stock adds a toasty, savory note and enhances the vegetables’ umami.
- No-stock shortcut: You can use plain water, but anchovy-kelp stock greatly improves the umami.
- Salt control: Since doenjang is salty, don’t add too much at first—taste after dissolving and add more if needed.
- Order of ingredients: Add potatoes → firm vegetables (onion) → soft vegetables (zucchini) → tofu to ensure everything cooks evenly.
- Storage/reheating: Doenjang-jjigae often tastes better after resting for a day. When reheating, the doenjang flavor can intensify; add a little water while reheating to dilute if it becomes too strong.
- Vegan version: Use kelp and shiitake for the stock instead of anchovies to make it vegetarian/vegan.
- Variations: Adding pork belly or seafood (clams) changes the broth’s character. If adding meat, fry it first to render fat for more flavor.

Enjoy cooking! If you’d like, I can also provide a low-sodium version, a pork-belly doenjang-jjigae recipe, or a clam (bashjig) doenjang-jjigae recipe.

In [ ]:
# recipe_chain
# translate_chain
# chain

## 사용자 함수를 Chain에 적용하기

### 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.
  - 보통 Lambda 표현식으로 정의한 함수를 LCEL chain에 포함시킬 때 사용한다.
  
#### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

### 함수 자체를 chain에 추가
- RunnableLambda에 전달할 함수 구문에 맞는 함수라면 RunnableLambda를 사용하지 않고 chain에 넣을 수 있다. 
- 단 함수로 정의하고 LCEL에 포함시켜야 한다. Lambda 표현식으로 작성한 함수는 `RunnableLambda`를 사용해야 한다.

### 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def plus(num1, num2):
    return num1 + num2

def wrapper_plus(num:list)->int:
    return plus(num[0], num[1])

chain = RunnablePassthrough() | wrapper_plus
chain = RunnablePassthrough() | \
    RunnableLambda(lambda num:plus(num[0], num[1]))
chain.invoke([10, 20])

30

In [ ]:
from langchain_core.runnables import chain
# @chain: 복잡한 체인구조를 정의한 함수를 Runnable로 만들때 사용.
# LCEL: 순차구조(흐름) 만 정의가 가능.
# @chain 함수를 이용해서 Chain구성: 조건문, 반복문을 이용해 chain의 흐름을
#                                 제어할 경우

@chain
def custom_chain(input_data:dict) -> dict[str, str]:
    # input_data: food, language, kor_recipe:bool
    # kor_recipe: True 한국어 레시피와 language 로 번역된 레시피 반환
    #           : False: language 로 번역된 레시피 반환
    # 조건절 -> LCEL 로는 구현이 안된.
    food = input_data['food']
    language = input_data['language']
    is_kor = input_data['kor_recipe']

    korean_recipe = recipe_chain.invoke({"food":food})
    
    result = translate_chain.invoke(
        {"content":korean_recipe, "language":language}
    )
    if is_kor:
        return (korean_recipe, result)
    else:
        return result
    
custom_chain


RunnableLambda(custom_chain)

In [33]:
result = custom_chain.invoke(
    {"food":"티본 스테이크", "language":"스페인어", "kor_recipe":True}
)
len(result)

2

In [36]:
result = custom_chain.invoke(
    {"food":"티본 스테이크", "language":"스페인어", "kor_recipe":False}
)
len(result)

7089

In [37]:
result

'# Bistec T-bone\n\n## Información básica de la receta\n- Dificultad: Media (es importante controlar el fuego y verificar la temperatura)\n- Tiempo de cocción: Preparación 40 minutos (incluye salado previo) + cocción 15–30 minutos (según grosor/método)\n- Raciones: Para 2 personas, tomando 400–600 g por persona (por ejemplo: 1,0–1,2 kg de T-bone para 2 personas)\n\n---\n\n## Ingredientes (para 2 personas)\n- 1 T-bone steak (se recomienda grosor de 3–5 cm; peso total 800 g–1,2 kg)\n- Sal gruesa (sal kosher) 1–1,5 cucharadas (suma para ambas caras)\n- Pimienta recién molida al gusto\n- Aceite de cocina (alto punto de humo: aceite de semilla de uva, de canola, etc.) 1 cucharada\n- Mantequilla sin sal 30 g\n- 3–4 dientes de ajo enteros (aplastar la parte superior)\n- Ramas frescas de tomillo y romero, 2–3 de cada una\n- (Opcional) Un poco de aceite de oliva, unas gotas de jugo de limón\n\nUtensilios: Sartén de hierro fundido (o sartén pesada), termómetro para carnes (altamente recomendado)

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [45]:
from langchain_community.cache import InMemoryCache, SQLiteCache
from langchain_core.globals import set_llm_cache

# set_llm_cache(InMemoryCache()) # 대화내역을 메모리에 저장.
set_llm_cache(SQLiteCache(database_path="cache.sqlite"))
# SQLite db이용. 디비파일 경로를 지정.

prompt = ChatPromptTemplate(
    [
        ("system", "당신은 유능한 assistant입니다. 질문에 20단어 이내로 답해주세요."),
        ("user", "{query}")
    ]
)
model = ChatOpenAI(model='gpt-5-mini')
chain = prompt | model | StrOutputParser()

In [46]:
chain.invoke({"query":"인공지능의 미래에 대해 알려줘."})

'AI는 지능화·자율화되어 경제·의료 혁신 촉진, 일자리 변화, 개인정보·윤리·규제 과제 발생.'

In [47]:
chain.invoke({"query":"인공지능의 미래에 대해 알려줘."})

'AI는 지능화·자율화되어 경제·의료 혁신 촉진, 일자리 변화, 개인정보·윤리·규제 과제 발생.'

In [ ]:
www.langchain.com